In [50]:
import pandas as pd
import os
import numpy as np
os.chdir("C:\\Users\\tim_s\\OneDrive\\Documenten")

## Importing data 

In [91]:
table=pd.read_excel("Big data table functionalities.xlsx")
table=table.set_index("Unnamed: 0")
del table.index.name
libraries=table.index
for i in table.columns:
    table.loc['Score',i]=table[i].sum()/4

## Calculator function to create a ranking for 1 parameter configuration

In [92]:
def calculator(num_features,weight_t,weight_u): 
    columns=['# top','# rest','Compute t','Compute u', 'weight t', 'weight u', 'score', 'rank']
    data = np.array([np.arange(4)]*8).T
    end_table= pd.DataFrame(data, index=libraries, columns=columns)
    
    top_table=table[table.columns[table.ix['Score'].argsort()[::-1][:num_features]]]
    bottom_table=table[table.columns[table.ix['Score'].argsort()[::-1][num_features:]]]
    bottom_table_2=bottom_table.transpose()
    
    for i in top_table.index:
        if i != 'Score':
            end_table.loc[i,'# top']=top_table.loc[i].sum()
            end_table.loc[i,'# rest']=bottom_table.loc[i].sum()
            end_table.loc[i,'Compute t']=end_table.loc[i,'# top']/num_features
            end_table.loc[i,'Compute u']=bottom_table_2[bottom_table_2[i]>0]['Score'].sum()/len(bottom_table.columns)
    end_table['weight t']=weight_t
    end_table['weight u']=weight_u
    end_table['score']=(end_table['Compute t']*end_table['weight t'])+(end_table['Compute u']*end_table['weight u'])
    end_table['rank']=ss.rankdata([-1 * i for i in end_table['score']]).astype(int)
    return end_table

## Function to create table with rankings of all parameter configurations

In [134]:
def create_final_table():
    idx = pd.MultiIndex.from_product([[5, 6, 7,8,9,10],['Email', 'mail-parser', 'Pyzmail','Flanker']],names=['Num top features', 'Library'])
    col = [0.5,0.6,0.7,0.8, 0.9]
    df = pd.DataFrame(0, idx, col)
    for i in list(df.index.levels)[0]:
        for j in df.columns:  
            temp=calculator(i,j,1-j)
            df.loc[i,j]=list(temp['rank'])
    return df